In [12]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                             _: 43.9 MiB
                            __: 43.9 MiB
                   docs_df_SE4: 43.9 MiB
                            _9: 43.9 MiB
                           _10: 43.9 MiB
                    embeddings: 14.9 MiB
           SentenceTransformer:  904.0 B
                          _i12:  587.0 B
                           _i4:  422.0 B
                           _i5:  422.0 B


- a software system to bring out information of web

- smart search engines can be considered unsupervised learning approaches, due to the nature of clustering related information without such label in hand

- Search Engines have evolved **from a text input and output service to** an experience that cuts across voice, video, documents, and conversations


- an **infinite problem** to solve


- **related** to information retrieval, language understanding


- the **value that an effective search tool can bring to a business is enormous**; a key piece of intellectual property. Often a search bar is the main interface between customers and the business. 
    - create a competitive advantage by delivering an improved user experience.
    


search engine popular approaches:
- manual implementation with dataframe + tf-idf
- Elastic Search + BM25
- BM25 + Azure Cognitive Search

Requirements:
- Search index for storing each document, reflecting relevant information and up to date information
    - data can be reorganized by date (suggestion)
- Query understanding
    - takes sentence and preprocessed data information **directly without much context**
    - we can extract words or tokens from the query to match **article_type** (suggestion)
        - query to match tags (done)
    - we can filter the search by either blog or News (suggestion)
        - or add multiple results available (blog, News, or both)
    - BM25 + Azure Cognitive search
- Query ranking
    - by consine similarity

## 1- Library and Data Imports

In [9]:
# Used to import data from S3.
import pandas as pd

# Used to create the dense document vectors.
import torch
from sentence_transformers import SentenceTransformer, util

# Used to create and store the Faiss index.
# import faiss
import numpy as np
# import pickle

# Used to do vector searches and display the results.
# from vector_engine.utils import vector_search, id2details

In [10]:
docs_df_SE4 = pd.read_csv('../Data/processed/SE_data4.csv')

In [11]:
# Instantiate the sentence-level DistilBERT
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens') 
# model = SentenceTransformer('all-MiniLM-L6-v2') 
# model = SentenceTransformer("multi-qa/-MiniLM-L6-cos-v1") 
# model = SentenceTransformer("ZeyadAhmed/AraElectra-Arabic-SQuADv2-QA") 

# Check if CUDA is available ans switch to GPU
if torch.cuda.is_available():
    model = model.to(torch.device("cuda"))
print(model.device)


cuda:0


In [12]:
# Convert abstracts to vectors
embeddings = model.encode(docs_df_SE4.content_clean.to_list(), show_progress_bar=True)

Batches: 100%|███████████████████████████████████████████████████████████████████████| 159/159 [00:18<00:00,  8.81it/s]


In [13]:
query = 'نبوي'.split()
print(query)
query_encoded = model.encode(query, show_progress_bar=True)
query_encoded = np.reshape(np.mean(query_encoded, axis=0), (-1, 1))

['نبوي']


Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 41.67it/s]


In [14]:
query_encoded.shape, embeddings.shape

((768, 1), (5084, 768))

In [16]:
cosine_scores = util.cos_sim(embeddings, query_encoded.T)
cosine_scores_ids = list(enumerate(np.reshape(cosine_scores, (cosine_scores.shape[0]))))
cosine_scores_sorted = sorted(cosine_scores_ids, key=lambda x: x[1], reverse=True) 

for id_, score in cosine_scores_sorted[:10]:
#     print(round(score, 2))
    print(docs_df_SE4.content_clean.values[id_])
    print(docs_df_SE4.url.values[id_])

حوار كاتب إبراهيم غرايبه كتاب أردن ممكن  
https://husna.fm/%D9%85%D9%86-%D9%85%D9%83%D8%AA%D8%A8%D8%A9-%D8%A7%D9%84%D8%A8%D9%84%D8%AF/%D8%AD%D9%88%D8%A7%D8%B1-%D9%85%D8%B9-%D8%A7%D9%84%D9%83%D8%A7%D8%AA%D8%A8-%D8%A5%D8%A8%D8%B1%D8%A7%D9%87%D9%8A%D9%85-%D8%BA%D8%B1%D8%A7%D9%8A%D8%A8%D8%A9-%D8%AD%D9%88%D9%84-%D9%83%D8%AA%D8%A7%D8%A8%D9%87-%D8%A7%D9%84%D8%A3%D8%B1%D8%AF%D9%86-%D8%A7%D9%84%D9%85%D9%85%D9%83%D9%86
حوار د يونس شناعه حديث سيره ذاتي عنوان شقاء سنه  
https://husna.fm/%D9%85%D9%86-%D9%85%D9%83%D8%AA%D8%A8%D8%A9-%D8%A7%D9%84%D8%A8%D9%84%D8%AF/%D8%AD%D9%88%D8%A7%D8%B1-%D9%85%D8%B9-%D8%AF%D9%8A%D9%88%D9%86%D8%B3-%D8%B4%D9%86%D8%A7%D8%B9%D8%A9%D8%8C-%D9%84%D9%84%D8%AD%D8%AF%D9%8A%D8%AB-%D8%AD%D9%88%D9%84-%D8%B3%D9%8A%D8%B1%D8%AA%D9%87-%D8%A7%D9%84%D8%B0%D8%A7%D8%AA%D9%8A%D9%87-%D8%A8%D8%B9%D9%86%D9%88%D8%A7%D9%86-%D8%B4%D9%82%D8%A7%D8%A1-%D8%A7%D9%84%D8%B3%D9%86%D9%8A%D9%86
قصه ملك حارس عجوز ليل شديد ب رود 
https://husna.fm/%D9%85%D8%A7%D8%B0%D8%A7-%D9%84%D9%88%D8%9F/%D9%82%D8%B5%D8

In [9]:
embeddings.shape, query_encoded.shape

((5084, 768), (768, 1))

In [10]:
result = np.dot(embeddings, query_encoded)
result_ids = list(enumerate(np.reshape(result, (result.shape[0]))))
result_ids_sorted = sorted(result_ids, key=lambda x: x[1], reverse=True)[200: 220]

In [11]:
for id_, score in result_ids_sorted:
    print(round(score, 2))
    print(docs_df_SE4.content_clean.values[id_])
#     print(docs_df_SE4.title.values[id_])

110.68
بترول وطني سرحان جفر حوى كميه قليل نفط مدير شركه بترول وطني محمد الخصاونه أن دراسه أول منطقه سرحان جفر أشار إلى تواجد كميه قليل نفط أضاف الخصاونه حسني أن عمر جيولوجي منطقه سرحان جفر أشبه إلى حد كبير عمر جيولوجي حقل ريش الخصاونه أن شركه هندي يونيفرسال انرجي قام عمل دراسه مسحيه منطقه سرحان جفر 2012 لافت إلى بدء تحليل دراسه حديث إذ ترتب حكومه أي كلف هذا دراسه أشار الخصاونه أن حقل أردني عمل آن حقل حمزه أزرق أنتج تراكميا إلى آن برميل نفط حقل ريش أنتج غايه آن ربع تريليون متر مكعب غاز تم آن دراسه منطقه سرحان جفر 
110.68
فصل تيار كهربائي منطقه شمال أعلن شركه كهرباء أربد نيه فصل تيار كهربائي اثنين منطقه محافظه إقليم شمال أربد جرش عجلون مفرق غايه صيانه تحسين جوده شبكه شركه بيان ل أن انقطاع شمل محافظ أربد منطقه صريح شرق محطه أبو عاشور جميع مشترك ثلاث فاز أن متوقع انقطاع ساعه تاسع صباح غايه ساعه حادي صباح إلى جانب منطقه ايدون جنوب مستشفى أربد تخصصي وقت متوقع انقطاع ساعه حادي صباح غايه واحد ظهر ما كان انقطاع منطقه ايدون جنوب دوار صحراء طريق قصر عواد ساعه واحد ظهر غايه ثالث عصر بين شركه أن فص

In [ ]:
# # Step 1: Change data type
# embeddings = np.array([embedding for embedding in embeddings]).astype("float32")

# # Step 2: Instantiate the index
# index = faiss.IndexFlatL2(embeddings.shape[1])

# # Step 3: Pass the index to IndexIDMap
# index = faiss.IndexIDMap(index)

# # Step 4: Add vectors and their IDs
# index.add_with_ids(embeddings, df.id.values)

# # Retrieve the 10 nearest neighbours
# D, I = index.search(np.array([embeddings[5415]]), k=10)

## References

- https://towardsdatascience.com/how-to-build-a-semantic-search-engine-with-transformers-and-faiss-dcbea307a0e8
